# Notes



Método da descida do gradiente aplicado a funções quadráticas
Tamanho do passo ótimo: $grad(f)^2/(grad(f)^TQgrad(f))$

In [6]:
# function [x, val_func] = MDMR_quadratic(Q, c, b, x0, epsilon)
# x = x0;
# k = 0;
# grad = Q * x + c;
# while (norm(grad) > epsilon)
# k = k+1;
# alpha = norm(grad)^2/(grad' * Q * grad);
# x = x - alpha * grad;
# val_func = 1/2 * x' * Q * x + c' * x + b;
# fprintf('iter %3d, normGrad=%2.6f, valFunc=%2.6f\n', k, norm(grad), valfunc);


import numpy as np

def MDMR_quadratic(Q, c, b, x0, epsilon):
    """
    Método do Gradiente Descendente para minimizar uma função quadrática.

    Args:
        Q: Matriz Q da função quadrática (deve ser simétrica e definida positiva).
        c: Vetor c da função quadrática.
        b: Escalar b da função quadrática.
        x0: Ponto inicial.
        epsilon: Tolerância para a norma do gradiente.

    Returns:
        x: Ponto mínimo encontrado.
        val_func: Valor da função no ponto mínimo.
    """
    
    x = x0
    k = 0
    grad = Q @ x + c

    while np.linalg.norm(grad) > epsilon:
        k += 1
        alpha = np.linalg.norm(grad)**2 / (grad.T @ Q @ grad)
        x = x - alpha * grad
        val_func = 0.5 * x.T @ Q @ x + c.T @ x + b
        print(f"iter {k:3d}, normGrad={np.linalg.norm(grad):2.6f}, valFunc={val_func:2.6f}")
        grad = Q @ x + c

    return x, val_func

# Exemplo: min f(x, y) = min (x^2 + 2*y^2), ponto inicial (2, 1) e tolerância 10^(-5)

Q = np.array([[2, 0], [0, 4]])
c = np.array([0, 0])
b = 0
x0 = np.array([2, 1])
epsilon = 0.000001

x_min, val_func_min = MDMR_quadratic(Q, c, b, x0, epsilon)

print(f"\nPonto mínimo encontrado: x = {x_min}")
print(f"Valor mínimo da função: f(x) = {val_func_min}")

# Aplicar o método da descida do gradiente com o passo otimizado ao problema quadrático: 
# min f(x_1, x_2) = (x_1 - 1)^2 + (x_1 - 1 - x_2)^2

# Reescrevendo na forma f(x) = 1/2 * x^T * Q * x + c^T * x + b

Q = np.array([[4, -2], [-2, 2]])
c = np.array([-4, 2])
b = 2
x0 = np.array([0, 0])
epsilon = 0.000001

x_min, val_func_min = MDMR_quadratic(Q, c, b, x0, epsilon)

print(f"\nPonto mínimo encontrado: x = {x_min}")
print(f"Valor mínimo da função: f(x) = {val_func_min}")

iter   1, normGrad=5.656854, valFunc=0.666667
iter   2, normGrad=1.885618, valFunc=0.074074
iter   3, normGrad=0.628539, valFunc=0.008230
iter   4, normGrad=0.209513, valFunc=0.000914
iter   5, normGrad=0.069838, valFunc=0.000102
iter   6, normGrad=0.023279, valFunc=0.000011
iter   7, normGrad=0.007760, valFunc=0.000001
iter   8, normGrad=0.002587, valFunc=0.000000
iter   9, normGrad=0.000862, valFunc=0.000000
iter  10, normGrad=0.000287, valFunc=0.000000
iter  11, normGrad=0.000096, valFunc=0.000000
iter  12, normGrad=0.000032, valFunc=0.000000
iter  13, normGrad=0.000011, valFunc=0.000000
iter  14, normGrad=0.000004, valFunc=0.000000
iter  15, normGrad=0.000001, valFunc=0.000000

Ponto mínimo encontrado: x = [ 1.39383439e-07 -6.96917194e-08]
Valor mínimo da função: f(x) = 2.914161449771316e-14
iter   1, normGrad=4.472136, valFunc=0.076923
iter   2, normGrad=0.344010, valFunc=0.002959
iter   3, normGrad=0.172005, valFunc=0.000114
iter   4, normGrad=0.013231, valFunc=0.000004
iter   5,

# Método de Newton

In [4]:
# Método de Newton
# Implementação do algoritmo

import numpy as np
from sympy import symbols, diff

def MN(f, grad, Hess, x0, epsilon):
    """
    Método de Newton para encontrar o mínimo de uma função.

    Args:
        f: Função a ser minimizada (numérica).
        grad: Função que calcula o gradiente da função f.
        Hess: Função que calcula a matriz Hessiana da função f.
        x0: Ponto inicial.
        epsilon: Tolerância para a norma do gradiente.

    Returns:
        x: Ponto mínimo encontrado.
        val_func: Valor da função no ponto mínimo.
    """
    x = x0
    valG = grad(x)
    valH = Hess(x)
    i = 0
    while (np.linalg.norm(valG) > epsilon) and (i < 10000):
        i += 1
        x = x - np.linalg.solve(valH, valG)
        val_func = f(x)
        print(f"k = {i}, f(x) = {val_func:.10f}")
        valG = grad(x)
        valH = Hess(x)
    
    if i == 10000:
        print("O método não convergiu")
    
    return x, val_func

def generate_functions(f):
    """
    Gera as funções de gradiente e Hessiana numericamente a partir da função simbólica f.

    Args:
        f: Função simbólica.

    Returns:
        gradient: Função que calcula o gradiente.
        hessian: Função que calcula a Hessiana.
    """
    # Definir variáveis simbólicas
    x, y = symbols('x y')
    
    # Calcular o gradiente
    grad_f = [diff(f, x), diff(f, y)]
    
    # Calcular a Hessiana
    hess_f = [[diff(grad_f[0], x), diff(grad_f[0], y)],
              [diff(grad_f[1], x), diff(grad_f[1], y)]]

    # Funções para calcular o gradiente e a Hessiana numericamente
    def gradient(point):
        return np.array([grad_f[0].subs({x: point[0], y: point[1]}),
                         grad_f[1].subs({x: point[0], y: point[1]})], dtype='float64')

    def hessian(point):
        return np.array([[hess_f[0][0].subs({x: point[0], y: point[1]}),
                          hess_f[0][1].subs({x: point[0], y: point[1]})],
                         [hess_f[1][0].subs({x: point[0], y: point[1]}),
                          hess_f[1][1].subs({x: point[0], y: point[1]})]], dtype='float64')

    return gradient, hessian

# Exemplo 1: min 100x^4 + 0.01y^4
x, y = symbols('x y')
f = 100 * x**4 + 0.01 * y**4

gradient, hessian = generate_functions(f)

# Ponto inicial e tolerância
x0 = np.array([1, 1])
epsilon = 1e-6

# Aplicar o método de Newton
def f_numeric(point):
    return float(f.subs({x: point[0], y: point[1]}))

x_min, val_func_min = MN(f_numeric, gradient, hessian, x0, epsilon)

print(f"\nExemplo 1 - Ponto mínimo encontrado: x = {x_min}")
print(f"Valor mínimo da função: f(x) = {val_func_min}")

# Exemplo 2: min sqrt(x^2 + 1) + sqrt(y^2 + 1)
f = (x**2 + 1)**0.5 + (y**2 + 1)**0.5

gradient, hessian = generate_functions(f)

# Ponto inicial e tolerância
x0 = np.array([1, 1])
epsilon = 1e-8

# Aplicar o método de Newton
x_min, val_func_min = MN(f_numeric, gradient, hessian, x0, epsilon)

print(f"\nExemplo 2 - Ponto mínimo encontrado: x = {x_min}")
print(f"Valor mínimo da função: f(x) = {val_func_min}")

# Exemplo 3: Outro exemplo com ponto inicial (0.5, 0.5)
x0 = np.array([0.5, 0.5])
x_min, val_func_min = MN(f_numeric, gradient, hessian, x0, epsilon)

print(f"\nExemplo 3 - Ponto mínimo encontrado: x = {x_min}")
print(f"Valor mínimo da função: f(x) = {val_func_min}")


ModuleNotFoundError: No module named 'sympy'